# 1. Setup
1. Import libraries
2. Import data
3. Basic data cleaning

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
# import numpy as np
import string
import difflib
%matplotlib inline

tn_df = pd.read_csv(
    '/Users/ronlodetti/Documents/Flatiron/1_phase/Project_1/Movie_Analysis_Project/data/imported/tn.movie_budgets.csv.gz')
conn = sqlite3.connect(
    '/Users/ronlodetti/Documents/Flatiron/1_phase/Project_1/Movie_Analysis_Project/data/imported/im.db')

# importing producers table
q = """
SELECT 
    movie_id,
    primary_title AS title,
    start_year AS year,
    runtime_minutes AS runtime,
    genres,
    primary_name AS producer
FROM movie_basics 
JOIN principals
    USING('movie_id')
JOIN persons
    USING('person_id')
WHERE job == 'producer' 
    AND start_year <= 2018 
    AND runtime_minutes BETWEEN 30 AND 240;

"""
imdb_df = pd.read_sql(q, conn)

## Feature Engineering

In [2]:
def clean_titles(series):
    series = series.translate(str.maketrans('', '', string.punctuation))
    series = series.replace(' ', '')
    series = series.lower()
    return series


def clean_currency(series):
    series = series.replace('$', '')
    series = series.replace(',', '')
    series = series.replace(' ', '')
    series = int(series)
    return series


def title_norm(df1, df2):
    '''
    This function takes the titles from df1, looks through the titles from
    df2, and if they pass a threshold, are replaced by matched title from 
    df2, aligning the titles for merging purposes. 
    '''
    df1.reset_index(drop=True, inplace=True)
    df2.reset_index(drop=True, inplace=True)
    for i in range(len(df1)):
        df1_title = df1['title'][i]
        match = difflib.get_close_matches(
            df1_title, df2['title'], n=1, cutoff=0.8)
        try:
            df2_title = match[0]
            index = df2[df2['title'] == df2_title].index[0]
            if (df1_title != df2_title) & (df1['year'][i] == df2['year'][index]):
                df1['title'].replace(df1_title, df2_title, inplace=True)
            else:
                continue
        except:
            continue

# Convert currencies to integers.
tn_df['production_budget'] = tn_df['production_budget'].apply(clean_currency)
tn_df['worldwide_gross'] = tn_df['worldwide_gross'].apply(clean_currency)

# Create a new column to calculate profit.
profit = tn_df['worldwide_gross'] - tn_df['production_budget']
tn_df['ROI'] = (profit/tn_df['production_budget']) * 100

# Extracting the year from 'release_date' column.
tn_df['year'] = pd.to_datetime(tn_df['release_date']).dt.year

# Filtering data for movies that have been released since 2006
tn_df = tn_df.loc[(tn_df['year'] >= 2006) & (tn_df['year'] <= 2018)]

# Norming and filtering column names
tn_df.rename(columns={'movie': 'title',
             'production_budget': 'budget'}, inplace=True)

imdb_df['title'] = imdb_df['title'].apply(clean_titles)
tn_df['title'] = tn_df['title'].apply(clean_titles)

imdb_df['genres'] = imdb_df['genres'].map(
    lambda x: x.split(','), na_action='ignore')
imdb_df = imdb_df.explode('genres')

title_norm(tn_df, imdb_df)

In [ ]:
# Write clean data to files
imdb_df.to_csv(
    '/Users/ronlodetti/Documents/Flatiron/1_phase/Project_1/Movie_Analysis_Project/data/cleaned/imdb_clean.csv',index=False)
tn_df.to_csv(
    '/Users/ronlodetti/Documents/Flatiron/1_phase/Project_1/Movie_Analysis_Project/data/cleaned/tn_clean.csv',index=False)

1. Find top and bottom producers
2. Explore genres
3. explore Runtime
4. explore budget


In [ ]:
q = """
SELECT *
FROM tn
JOIN imdb
    USING('title','year')
JOIN producers
USING(movie_id);

"""
df = pd.read_sql(q, conn)

In [ ]:
prod = list(df['producer'].value_counts()[df['producer'].value_counts()>7].index)

df2 = df[df['producer'].isin(prod)][['producer','ROI','budget','title','movie_id']].drop_duplicates(['movie_id','producer'])

df2.groupby('producer').agg(['mean','median','count'])['ROI'].sort_values('mean',ascending=False)


In [ ]:
df_single = df[['producer','ROI','budget','title','movie_id']].drop_duplicates(['movie_id','producer'])

prod = list(df_single['producer'].value_counts()[df_single['producer'].value_counts()>3].index)

df2 = df_single[df_single['producer'].isin(prod)]

mn = df2.groupby('producer').agg(['mean','median','count'])['ROI'].sort_values('mean',ascending=False)
mdn = df2.groupby('producer').agg(['mean','median','count'])['ROI'].sort_values('median',ascending=False)
top_mn = list(mn.head(10).index)
bot_mn = list(mn.tail(10).index)
top_mdn = list(mdn.head(10).index)
bot_mdn = list(mdn.tail(10).index)

In [ ]:
top_mn

In [ ]:
top_mdn

In [ ]:
bot_mn

In [ ]:
bot_mdn

In [ ]:
q = """
SELECT *
FROM tn
JOIN imdb
    USING('title','year')
JOIN producers
USING(movie_id)
WHERE producer in ('Andrew Macdonald',
 'Mark Johnson',
 'Michael Costigan',
 'Ridley Scott',
 'George Furla',
 'René Besson',
 'Randall Emmett',
 'Lars Knudsen',
 'Celine Rattray',
 'Cassian Elwes');

"""
bot = pd.read_sql(q, conn).drop_duplicates(['movie_id'])

In [ ]:
q = """
SELECT *
FROM tn
JOIN imdb
    USING('title','year')
JOIN producers
USING(movie_id)
WHERE producer in ('James Wan',
 'Peter Safran',
 'Janet Healy',
 'Andrew Form',
 'Brad Fuller',
 'Jason Blum',
 'Michael Bay',
 'Christopher Meledandri',
 'Iain Canning',
 'Emile Sherman');

"""
top = pd.read_sql(q, conn).drop_duplicates(['movie_id'])

In [ ]:
top['budget'].median()

In [ ]:
for i in [top,bot]:
    num = i['month'].value_counts(normalize=True)
    
    print(num)
    print("\n")

In [ ]:
bot.columns

In [ ]:
q = """
SELECT *
FROM opus
JOIN imdb
    USING('title','year')
JOIN producers
USING(movie_id)
WHERE producer in ('Andrew Macdonald',
 'Mark Johnson',
 'Michael Costigan',
 'Ridley Scott',
 'George Furla',
 'René Besson',
 'Randall Emmett',
 'Lars Knudsen',
 'Celine Rattray',
 'Cassian Elwes');

"""
test = pd.read_sql(q, conn).drop_duplicates(['movie_id'])
len(test)